In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
headers = {"User-Agent": user_agent}

In [3]:
response = requests.get(url=f"https://myskin.md/brendy", headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

brand_urls = []
for brand_tag in soup.find_all("a", class_="brand-name"):
    brand_href = brand_tag.get("href")
    brand_urls.append(f"https://myskin.md{brand_href}")

print(f"Brands: {len(brand_urls)}")

Brands: 56


In [5]:
products = []

for brand_url in brand_urls:
    response = requests.get(url=brand_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    total_pages = 1

    paginator = soup.find("div", class_="paginator_wrapper")

    if paginator:
        total_pages = int(paginator.get("data-pages"))

    for page_id in range(total_pages):
        response = requests.get(url=f"{brand_url}?page={page_id}", headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        if response.status_code != 200:
            if response.status_code == 404:
                print(f"Exit! Brand: {brand_url} Page #{page_id}: 404 error.")
                break
            else:
                print(
                    f"Brand: {brand_url} Page #{page_id} status_code: {response.status_code}"
                )
                continue

        for product in soup.find_all("div", class_="product-block"):
            title = product.find("a", class_="title").text.strip()
            href = product.find("a", class_="title")["href"]
            img_src = product.find("img")["src"]
            curr_price = (
                float(product.find("span", class_="new-price").text.strip(" MDL"))
                if product.find("span", class_="new-price")
                else None
            )

            old_price = (
                float(product.find("span", class_="old-price").text.strip(" MDL"))
                if product.find("span", class_="old-price")
                else None
            )

            if not curr_price:
                curr_price = (
                    float(product.find("span", class_="price").text.strip(" MDL"))
                    if product.find("span", class_="price")
                    else None
                )

            products.append(
                {
                    "brand": brand_url,
                    "title": title,
                    "link": f"https://myskin.md{href}",
                    "image": img_src,
                    "curr_price": curr_price,
                    "old_price": old_price,
                }
            )

print(len(products))

1778
